In [2]:

from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader

import bs4
loader=WebBaseLoader(web_paths=("https://www.taniarascia.com/getting-started-with-react/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

documents=loader.load()
documents

[Document(page_content='I\'ve been hearing about React since I first started learning JavaScript, but I\'ll admit I took one look at it and it scared me. I saw what looked like a bunch of HTML mixed with JavaScript and thought, isn\'t this what we\'ve been trying to avoid? What\'s the big deal with React?\nInstead, I focused on just learning vanilla JavaScript and working with jQuery in a professional setting. After a few frustrated, failed attempts to get started with React, I finally started to get it, and I began to see why I might want to use React instead of vanilla JS or jQuery.\nI tried to condense everything I\'ve learned into a nice introduction to share with you, so here it is.\nPrerequisites\nThere are a few things you should know in advance before you start playing around with React. If you\'ve never used JavaScript or the DOM at all before, for example, I would get more familiar with those before trying to tackle React.\nHere are what I consider to be React prerequisites.\

In [3]:

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:30],OllamaEmbeddings())

In [4]:
from langchain_community.llms import Ollama #using an open source model 
llm=Ollama(model="llama2")
llm

Ollama()

In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""answer the question based on context only
                                        <context>
                                        {context}
                                        </context> 
                                          Question;{input}""")


In [8]:
!pip install -U langchain langchain-community


  Using cached langchain_community-0.2.1-py3-none-any.whl.metadata (8.9 kB)
Using cached langchain_community-0.2.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.33
    Uninstalling langchain-community-0.0.33:
      Successfully uninstalled langchain-community-0.0.33


In [6]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
retriever=db.as_retriever()


In [7]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)


In [13]:
response=retrieval_chain.invoke({"input":"create an objective question from the context given. question should have 4 options out of which 1 is correct "})
print(response['answer'])


Context:
React is a popular JavaScript library used for building user interfaces. It can be used to create interactive and responsive web applications. In this article, we'll cover the basics of React, including components, state, props, working with form data, pulling data in from an API, and deploying an app.

Question:
What is the purpose of using the `componentDidMount()` method in a React component?

A) To update the component's state when the component mounts
B) To render the component when the component mounts
C) To handle events when the component mounts
D) To create a new instance of the component when the component mounts

Correct answer: A) To update the component's state when the component mounts
